In [12]:
from dolfin import *
from dolfin.fem.assembling import _create_dolfin_form
import ffc
import ufl
mesh = UnitIntervalMesh(10)
#mesh = UnitSquareMesh(10,10)
V = FunctionSpace(mesh, "Hermite", 3)
w = TestFunction(V)
Pv = TrialFunction(V)
a = ufl.inner(w, Pv)*dx
#%debug -b /home/fenics/local/lib/python2.7/site-packages/ffc/compiler.py:188 ffc.jit(a)
#%debug -b /home/fenics/local/lib/python2.7/site-packages/ffc/evaluatedof.py:189 ffc.jit(a)
_ = ffc.jit(a)

  Calling FFC just-in-time (JIT) compiler, this may take some time.
    MBD: Check whether hermite actually needs oriented jacobian.


In [13]:
from ufl import *
from ffc import *

e = FiniteElement("Hermite", triangle, 3)
plot(e)

  Unable to plot element, Soya3D not available (install package python-soya).


## Q 9248
[Linear elasticity problems on domains with different materials](https://fenicsproject.org/qa/9248/linear-elasticity-problems-domains-with-different-materials?show=9248)

The solution after incorporating all the suggestions in the anwers is constantly zero... why?

In [ ]:
from __future__ import print_function
from dolfin import *
from mshr import *
import sys, math, numpy

# Create classes for defining parts of the boundaries and the interior
# of the domain
class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[1], 0.0)

class Top(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (between(x[1], (0.5, 1.1)))

class Obstacle(SubDomain):
    def inside(self, x, on_boundary):
        return (x[0]*x[0] + x[1]*x[1] <= 0.25) 

# Define parameters
r = 1.0
MeshDensity = 20

# Initialize sub-domain instances
top = Top()
bottom = Bottom()
obstacle = Obstacle()

# Define the domain and generate the mesh
domain = Circle(Point(0, 0), r) - Rectangle(Point(-1.0, -2.0), Point(1.0, 0.0))
mesh = generate_mesh(domain,MeshDensity,"cgal")
#plot(mesh, 'Mesh')

# Initialize mesh function for interior domains
domains = CellFunction("size_t", mesh)
domains.set_all(0)
obstacle.mark(domains, 1)
#print(domains.array())

# Initialize mesh function for boundary domains
boundaries = FacetFunction("size_t", mesh)
boundaries.set_all(0)
bottom.mark(boundaries, 0)
top.mark(boundaries, 1)
#print(boundaries.array())

# Define input data
a1 = Constant(1.0)
a0 = Constant(0.01)
g = Constant((0.0, -1.0))
f = Constant((0.0, 1.0))

# Define function space and basis functions
V = VectorFunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

# Define Dirichlet boundary ( x=0, y=0 )
c = Constant((0,0))

# Define Dirichlet boundary conditions at top and bottom boundaries
bcs = DirichletBC(V, c, boundaries, 0)

# Define new measures associated with the interior domains and
# exterior boundaries
dx = Measure('dx', domain=mesh, subdomain_data=domains)
ds = Measure('ds', domain=mesh, subdomain_data=boundaries)

# Define variational form
F = (inner(a0*grad(u), grad(v))*dx(0) + (inner(a1*grad(u), grad(v)))*dx(1)
     - inner(g, v)*ds(1)
     - inner(f, v)*dx(0) - inner(f, v)*dx(1))

# Separate left and right hand sides of equation
a, L = lhs(F), rhs(F)

# Solve problem
u_h = Function(V)
solve(a == L, u_h, bcs)

In [14]:
import numpy as np
np.max(np.abs(u_h.compute_vertex_values(mesh)))

0.0

# Blah

In [8]:
from dolfin import *
mesh = UnitIntervalMesh(10)
V = FunctionSpace(mesh, 'Hermite', 3)
f = Argument(V, 0)
R = assemble(f*dx)